Reference: https://medium.com/@rami.krispin/setting-a-natural-language-to-sql-code-generator-with-python-d267f40d7218

Current issue is that while we have successfully tested our Bedrock GenAI PoC RAG model (with Antjropic CLuade LLM) or the LDFE job status report to query around forecast anomalies in natural language, there is a tendency to hallucinate in the response (even with temperature 0 and 1) and fundamental questions such as hoe many forecasts were run last night, how many succeeded, failed etc. with or without a specific error message or anomaly are not satisfactory.

This notebook is to test how to: query in natural lanaguage the job status report, then convert into a SQL query (potentially using AI or GenAI with an LLM) to return the desired query result

In [1]:
import os

import mysql.connector

import pandas as pd

import create_sql_prompt as csp

import os

import openai

In [2]:
local_host = os.getenv("RDS_HOST")
local_user = os.getenv("RDS_USER")
local_pwd = os.getenv("RDS_PASSWORD")

In [3]:
conn = mysql.connector.connect(host=local_host, 
                               user=local_user, 
                               password=local_pwd, 
                               database="rr_core")

#### Create SQL Prompt

In [4]:
# *check* below needs to understand fail or failed

# BELOW QUESTIONS THAT SHOULD BE ANSWERABLE

# query = "Hello, how are you ?"

# query = "How many inference jobs fail on 14th Jun 2024 ?"

query = "How many inference jobs failed this morning"
# query = "What is the main anomaly in the data"
# query = "What was the main anomaly in inference jobs today"
# query = "What has been the main forecast error recently"

# query = "What % of jobs succeeded versus failed in the last 7 days"
# query = "What has been the trend in failed v. successful runs"


In [5]:
# formulate OpenAI prompt
prompt = csp.create_message(conn, query = query)

/Users/barry.walsh/rotaready/rr_repos/ML-LabourDemandForecasting/notebooks/GenAI/create_sql_prompt.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tbl_describe = pd.read_sql_query("select * from ldfe_jobs", con=conn)


In [6]:
print(prompt.system)


    Given the following SQL table, your job is to write queries given a user's request. 

    Please consider verbs with all their tenses and any dates mentioned default to this year unless specified.

    CREATE TABLE `ldfe_jobs` (`id` int(11) unsigned NOT NULL AUTO_INCREMENT,
                    `timestamp` timestamp NOT NULL DEFAULT NULL,
                    `realm` varchar(40) NOT NULL,
                    `site` varchar(4) COLLATE utf8mb4_unicode_ci NOT NULL,
                    `jobtype` varchar(40) NOT NULL,
                    `status` ENUM('success', 'fail'),
                    `note` varchar(500) DEFAULT NULL,
                    `fail reason` varchar(500) DEFAULT NULL,
                    PRIMARY KEY (`id`)) 

    


In [7]:
# transform prompt into OpenAI format
converted_message = [
    {
      "role": "system",
      "content": prompt.system
    },
    {
      "role": "user",
      "content": prompt.user
    }
    ]

#### OpenAI API

In [8]:
openai.api_key = os.getenv("OPENAI_API_KEY")


In [9]:
response = openai.chat.completions.create(
        model = "gpt-3.5-turbo",
        messages = converted_message,
        temperature = 0.3,
        max_tokens = 256)

In [10]:
print(response)

ChatCompletion(id='chatcmpl-9bQlrbNcxghaMzzPz6gNGx4OrSSKZ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="```sql\nSELECT COUNT(*) AS num_failed_jobs\nFROM ldfe_jobs\nWHERE DATE(timestamp) = CURDATE()\nAND status = 'fail'\n```", role='assistant', function_call=None, tool_calls=None))], created=1718708071, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=31, prompt_tokens=182, total_tokens=213))


In [11]:
# extract sql query from response
sql = response.choices[0].message.content
sql

"```sql\nSELECT COUNT(*) AS num_failed_jobs\nFROM ldfe_jobs\nWHERE DATE(timestamp) = CURDATE()\nAND status = 'fail'\n```"

#### Parse SQL query

In [12]:
sql = sql.replace("```", "") # get rid of ``` in query
sql

"sql\nSELECT COUNT(*) AS num_failed_jobs\nFROM ldfe_jobs\nWHERE DATE(timestamp) = CURDATE()\nAND status = 'fail'\n"

In [13]:
sql = sql[4:] # remove first 5 characters i.e. sql\n
sql

"SELECT COUNT(*) AS num_failed_jobs\nFROM ldfe_jobs\nWHERE DATE(timestamp) = CURDATE()\nAND status = 'fail'\n"

In [14]:
# test the OpenAI generated SQL query
# 
result = pd.read_sql_query(sql, con=conn)
result

/var/folders/t6/ysrkn8dn65b6bsvj1sb28p9m0000gp/T/ipykernel_41899/2713898106.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(sql, con=conn)


,num_failed_jobs
0,83


#### Follow-on tests

#### BACKUP

In [ ]:
query = """select count(*) 
                from ldfe_jobs 
                where timestamp >= DATE_SUB(timestamp(current_date), INTERVAL 0 DAY);
        """

result = pd.read_sql_query(query, con=conn)
result

In [ ]:
conn.close()

#### S3 - may need to test later

In [ ]:
s3 = boto3.client('s3', 'eu-west-1')

    s3bucket = 'rotaready-machine-learning'
    
    # create dataframe for job status
    if job_start == True:
        status_cols= ['timestamp', 'realm', 'site', 'jobtype']
        s3key = 'ldfe/debug/inference/job_start.csv'
    else: # job status report
        status_cols= ['timestamp', 'realm', 'site', 'jobtype', 'status', 'note', 'fail reason']
        # *check* the job status report should really only be exported after last client/site pair run, not after each of them
        s3key = 'ldfe/debug/inference/job_status.csv'

    status_df = pd.DataFrame([job_data], columns = status_cols)

    # get existing data from S3
    obj = s3.get_object(Bucket=s3bucket, Key=s3key)
    current_data = pd.read_csv(BytesIO(obj['Body'].read())) 